In [1]:
import pandas as pd


In [16]:
filepath = r"https://github.com/data-to-insight/ERN-sessions/raw/main/data/903_xlsx_errors.xlsx"

dfs = pd.read_excel(filepath, sheet_name=None)

dfs.keys()

#dfs['header'].info()

dict_keys(['ad1', 'episodes', 'header', 'missing', 'oc2', 'oc3', 'placed_for_adoption', 'previous_permenance', 'reviews', 'uasc'])

In [5]:
import openpyxl

In [28]:
#.notna()

header = dfs['header']
header.head()

header['DOB'] = pd.to_datetime(header['DOB'], format="%d/%m/%Y", errors='coerce')



In [33]:
header = dfs['header']
 
# header.info()
# header.head() # its' d/m/y
 
header['DOB'] = pd.to_datetime(header['DOB'], format="%d/%m/%Y", errors='coerce')
 
 
#invalid_dobs = header['DOB'].notna()
#error_rows = header[~invalid_dobs]
 
#invalid_dobs = header['DOB'].isna()
#error_rows = header[invalid_dobs]
 


In [ ]:
# 115 - Date of Local Authority's (LA) decision that a child should be placed for adoption is not a valid date."
# table is placed_for_adoption
# column is DATE_PLACED

In [38]:
p4a = dfs['placed_for_adoption']

p4a.info()

p4a['DATE_PLACED'] = pd.to_datetime(p4a['DATE_PLACED'], format="%d/%m/%Y", errors='coerce')

p4a.info() 

invalid_date = pfa['d']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   CHILD                 30 non-null     object        
 1   DOB                   30 non-null     object        
 2   DATE_PLACED           29 non-null     datetime64[ns]
 3   DATE_PLACED_CEASED    30 non-null     object        
 4   REASON_PLACED_CEASED  30 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 1.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   CHILD                 30 non-null     object        
 1   DOB                   30 non-null     object        
 2   DATE_PLACED           29 non-null     datetime64[ns]
 3   DATE_PLACED_CEASED    30 non-null   

KeyError: 'd'

In [39]:
# 115 - Date of Local Authority's (LA) decision that a child should be placed for adoption is not a valid date."
# table is placed_for_adoption
p4a = dfs['placed_for_adoption']
# column is DATE_PLACED
p4a['DATE_PLACED'] = pd.to_datetime(p4a['DATE_PLACED'], format="%d/%m/%Y", errors="coerce")
 
invalid_date = p4a['DATE_PLACED'].isna()
 
error_rows = p4a[invalid_date]
error_rows

,CHILD,DOB,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED
29,bad_entry_1,Not a date,NaT,This also isn't a date,oop


In [40]:
ethncity_codes = [
        "WBRI",
        "WIRI",
        "WOTH",
        "WIRT",
        "WROM",
        "MWBC",
        "MWBA",
        "MWAS",
        "MOTH",
        "AIND",
        "APKN",
        "ABAN",
        "AOTH",
        "BCRB",
        "BAFR",
        "BOTH",
        "CHNE",
        "OOTH",
        "REFU",
        "NOBT",
    ]
 
valid_ethnicity = header['ETHNIC'].isin(ethncity_codes)
 
error_rows = header[~valid_ethnicity]
 
error_rows
 
# 104 - Data entry to record the status of former carer(s) of an adopted child is invalid.
#code_list = ["0", "1"]

,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB
2675,bad_entry_1,7,NaT,Alien,UPDOG,NaN,NaN


In [ ]:
former_carer = ["0", "1"]

valid_carer = p4a['']

In [41]:
# 114 - Data entry to record the status of former carer(s) of an adopted child is invalid.
# 114 - Data entry to record the status of former carer(s) of an adopted child is invalid.
# Whenw e do an isin we need to check the datatype of the list is the same as the values
# valid codes
code_list = ["0", "1"]
 
# table is ad1
ad1 = dfs['ad1']
 
ad1['FOSTER_CARE'] = ad1['FOSTER_CARE'].astype('str')
 
# column is FOSTER_CARE
valid_codes = ad1['FOSTER_CARE'].isin(code_list)
 
error_rows = ad1[~valid_codes]
 
error_rows

,CHILD,DOB,DATE_INT,DATE_MATCH,FOSTER_CARE,NB_ADOPTR,SEX_ADOPTR,LS_ADOPTR
29,bad_entry,datedate,notdate,datenot,777,Q,Will,let's go


In [42]:
# 184 - Date of decision that a child should be placed for adoption is before the child was born.
 
df = pd.merge(p4a, header, how='left', on='CHILD', suffixes=[None, "_header"])
 
df['DOB'] = pd.to_datetime(df['DOB'], format="%d/%m/%Y", errors='coerce')
df['DATE_PLACED'] = pd.to_datetime(df['DATE_PLACED'], format="%Y-%m-%d", errors='coerce')
 
placed_before_birth = (df['DATE_PLACED'] >= df['DOB']) | df['DATE_PLACED'].isna()
 
error_rows = df[~placed_before_birth]
 
error_rows
 
# 302 - First episode starts before child was born.
# tables are header and episodes
# DOB from header and DECOM from episodes


,CHILD,DOB,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED,SEX,DOB_header,ETHNIC,UPN,MOTHER,MC_DOB


In [43]:
# 302 - First episode starts before child was born.
# tables are header and episodes
# DOB from header and DECOM from episodes
 
episodes = dfs['episodes']
 
episodes['DECOM'] = pd.to_datetime(episodes['DECOM'], format="%Y-%m-%d", errors='coerce')
 
# sorts values
episodes = episodes.sort_values(['CHILD', 'DECOM'], ascending=True)
 
# drops duplicates
episodes.drop_duplicates('CHILD', keep='first', inplace=True)
 
df = pd.merge(episodes, header, how='left', on='CHILD', suffixes=['_epi', '_hdr'])
 
error_rows = df[df['DECOM'] < df['DOB']]
error_rows

,CHILD,DECOM,RNE,LS,CIN,PLACE,PLACE_PROVIDER,DEC,REC,REASON_PLACE_CHANGE,HOME_POST,PL_POST,URN,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB


In [46]:
# 188 - Child is aged under 4 years at the end of the year but a Strengths and Difficulties (SDQ) score or a reason for no SDQ score has been completed
oc2 = dfs['oc2']
 
oc2['DOB'] = pd.to_datetime(oc2['DOB'], format="%d/%m/%Y", errors='coerce')
 
collection_end = pd.to_datetime('31/03/2020', dayfirst=True)
 
oc2['4bday'] = oc2['DOB'] + pd.DateOffset(years=4)
 
error_mask = (oc2['4bday'] > collection_end) & (oc2['SDQ_SCORE'].notna() | oc2['SDQ_REASON'].notna())
 
error_rows = oc2[error_mask]
 
error_rows
 
# 189 - Child is aged 17 years or over at the beginning of the year, but an Strengths
#  and Difficulties (SDQ) score or a reason for no Strengths and Difficulties (SDQ) score has been completed.
 
# 01/04/2020
# age is 17

,CHILD,DOB,SDQ_SCORE,SDQ_REASON,CONVICTED,HEALTH_CHECK,IMMUNISATIONS,TEETH_CHECK,HEALTH_ASSESSMENT,SUBSTANCE_MISUSE,INTERVENTION_RECEIVED,INTERVENTION_OFFERED,4bday
2675,bad_child,2024-12-05,oops,bad,no,yes,maybe,i,don't,know,possibly,yes,2028-12-05


In [47]:
# 189 - Child is aged 17 years or over at the beginning of the year, but an Strengths
#  and Difficulties (SDQ) score or a reason for no Strengths and Difficulties (SDQ) score has been completed.
 
collection_start = pd.to_datetime('01/04/2019', dayfirst=True)
 
over_17_start = oc2['DOB'] + pd.DateOffset(years=17) < collection_start
sdq_score_reason = oc2['SDQ_SCORE'].notna() | oc2['SDQ_REASON'].notna()
 
error_mask = over_17_start & sdq_score_reason
 
error_rows = oc2[error_mask]
error_rows

,CHILD,DOB,SDQ_SCORE,SDQ_REASON,CONVICTED,HEALTH_CHECK,IMMUNISATIONS,TEETH_CHECK,HEALTH_ASSESSMENT,SUBSTANCE_MISUSE,INTERVENTION_RECEIVED,INTERVENTION_OFFERED,4bday
20,947046,2001-08-22,NaN,SDQ5,0,1,0,1,0,1,0,0,2005-08-22
33,714655,2001-10-01,NaN,SDQ2,1,1,0,1,1,0,1,0,2005-10-01
35,819792,2001-04-14,25,NaN,1,1,1,1,0,1,1,1,2005-04-14
43,197474,2001-03-30,5,NaN,0,1,1,1,0,1,1,0,2005-03-30
58,552166,2001-11-09,34,NaN,1,1,1,1,1,1,1,1,2005-11-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2634,97512,2001-12-27,NaN,SDQ1,0,0,1,0,1,0,0,1,2005-12-27
2653,413076,2001-05-14,32,NaN,1,0,1,0,0,0,1,1,2005-05-14
2659,482205,2001-04-26,32,NaN,1,0,0,0,0,0,0,0,2005-04-26
2660,473732,2001-06-17,NaN,SDQ1,0,1,0,1,0,1,0,0,2005-06-17
